In [ ]:
!pip install dash
!pip install dash-bootstrap-components
!pip install dash-html-components


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 13.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install jupyter_dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.6 MB/s eta 0:00:00


In [21]:
from jupyter_dash import JupyterDash

In [25]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.colors as mcolors
import base64

commute_flows = pd.read_csv('/content/Commute Flows1.csv')
commute_flows['County Name Residence'] = commute_flows['County Name Residence'].apply(lambda x: 'From ' + x)

county_to_borough = {
    'From New York County': 'From Manhattan',
    'From Kings County': 'From Brooklyn',
    'From Bronx County': 'From Bronx',
    'From Queens County': 'From Queens',
    'New York County': 'Manhattan',
    'Kings County': 'Brooklyn',
    'Bronx County': 'Bronx',
    'Queens County': 'Queens'
}

commute_flows = commute_flows[~commute_flows['County Name Residence'].isin(['From Richmond County'])]
commute_flows = commute_flows[~commute_flows['County Name Workplace'].isin(['Richmond County'])]

commute_flows.replace({'County Name Residence': county_to_borough}, inplace=True)
commute_flows.replace({'County Name Workplace': county_to_borough}, inplace=True)

all_nodes = pd.concat([commute_flows['County Name Residence'], commute_flows['County Name Workplace']]).unique()

colors = ['LightSkyBlue', 'PaleGreen', 'LightCoral', 'LemonChiffon', 'Thistle']
colors_rgba = [mcolors.to_rgba(color, alpha=0.8) for color in colors]

node_colors = [f'rgba({int(color[0]*255)},{int(color[1]*255)},{int(color[2]*255)},0.8)' 
               for color in colors_rgba for _ in range(2)]

node_dict = {node: index for index, node in enumerate(all_nodes)}

color_dict = {node: color for node, color in zip(all_nodes, node_colors)}

link_colors = [color_dict[commute_flows['County Name Residence'].iloc[i]] for i in range(len(commute_flows))]

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_nodes,
        color=node_colors
    ),
    link=dict(
        source=commute_flows['County Name Residence'].map(node_dict).values,
        target=commute_flows['County Name Workplace'].map(node_dict).values,
        value=commute_flows['Workers in Commuting Flow'].values,
        color=link_colors
    ))])

fig.update_layout(font=dict(
    family="Arial Black",
    size=20,
    color="black"
))

app = JupyterDash(__name__)

def encode_image(image_file):
    encoded = base64.b64encode(open(image_file, 'rb').read())
    return 'data:image/png;base64,{}'.format(encoded.decode())



import dash_bootstrap_components as dbc
import dash_html_components as html

# Set bootstrap theme
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css"])


app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("NYC Commute Dashboard", className="text-center", style={'font-family': 'Arial', 'font-size': '36px'}), width=12)
    ], style={'margin-bottom': '20px'}),
    
    dbc.Row([
        dbc.Col([
            html.H2("Sankey diagram showing the daily flow of workers for NYC boroughs (excluding Staten Island)", id="sankey-title", className="text-center", style={'cursor': 'help'}),
            dcc.Graph(figure=fig, style={'height': '600px', 'width': '600px', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}, id="sankey-graph"),
            html.I(className="fas fa-info-circle fa-lg", id="sankey-icon", style={'margin-left': '5px', 'color': 'black', 'cursor': 'help'}),
            dbc.Tooltip("This Sankey diagram visualizes the daily flow of workers between different boroughs in NYC. It illustrates the number of workers commuting from one borough to another. The thickness of the links represents the magnitude of commuting flow.", target="sankey-icon")
        ], width=12)
    ], style={'border': '2px black solid', 'padding': '10px', 'margin-top': '10px', 'margin-bottom': '20px', 'text-align': 'center'}),
    
    dbc.Row([
        dbc.Col([
            html.H2("Proposed Route of Interborough Express for the underserved yet high transit use parts of Brooklyn and Queens", id="route-title", className="text-center", style={'cursor': 'help'}),
            html.Img(src=encode_image("New Line.png"), style={'height':'50%', 'width':'50%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}),
            html.I(className="fas fa-info-circle fa-lg", id="route-icon", style={'margin-left': '5px', 'color': 'black', 'cursor': 'help'}),
            dbc.Tooltip("This graph depicts the proposed route of the Interborough Express, a transit service aimed at improving transportation in underserved areas of Brooklyn and Queens. It highlights the potential route and key stops.", target="route-icon")
        ], width=12)
    ], style={'border': '2px black solid', 'padding': '10px', 'margin-top': '10px', 'margin-bottom': '20px', 'text-align': 'center'}),
    
    dbc.Row([
        dbc.Col([
            html.H2("Pie Charts showing transit method used to commute for different parts of the borough", id="pie-title", className="text-center", style={'cursor': 'help'}),
            dbc.Card([
                dbc.CardBody([
                    html.Img(src=encode_image("bronx pie chart.png"), style={'height':'50%', 'width':'50%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}),
                    html.I(className="fas fa-info-circle fa-lg", id="bronx-icon", style={'margin-left': '5px', 'color': 'red', 'cursor': 'help'}),
                    dbc.Tooltip("This pie chart presents the distribution of transit methods used by commuters in Bronx. It showcases the percentage of commuters using different modes of transportation, such as public transit, car, walking, etc.", target="bronx-icon")
                ])
            ], style={'border': '2px solid red', 'margin-bottom': '10px'})
        ], width=12)
    ], style={'border':'2px black solid', 'padding': '10px', 'margin-top': '10px', 'margin-bottom': '20px', 'text-align': 'center'}),
    
    dbc.Row([
        dbc.Col([
            html.H2("Pie Charts showing transit method used to commute for different parts of the borough", id="pie2-title", className="text-center", style={'cursor': 'help'}),
            dbc.Card([
                dbc.CardBody([
                    html.Img(src=encode_image("Queens Pie Chart.png"), style={'height':'50%', 'width':'50%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}),
                    html.I(className="fas fa-info-circle fa-lg", id="queens-icon", style={'margin-left': '5px', 'color': 'blue', 'cursor': 'help'}),
                    dbc.Tooltip("This pie chart showcases the distribution of transit methods used by commuters in Queens. It visualizes the proportion of commuters using various transportation modes, such as public transit, car, walking, etc.", target="queens-icon")
                ])
            ], style={'border': '2px solid blue', 'margin-bottom': '10px'})
        ], width=12)
    ], style={'border': '2px black solid', 'padding': '10px', 'margin-top': '10px', 'margin-bottom': '20px', 'text-align': 'center'}),
    
    dbc.Row([
        dbc.Col([
            html.H2("Pie Charts showing transit method used to commute for different parts of the borough", id="pie3-title", className="text-center", style={'cursor': 'help'}),
            dbc.Card([
                dbc.CardBody([
                    html.Img(src=encode_image("Brooklyn Pie Chart.png"), style={'height':'50%', 'width':'50%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}),
                    html.I(className="fas fa-info-circle fa-lg", id="brooklyn-icon", style={'margin-left': '5px', 'color': 'green', 'cursor': 'help'}),
                    dbc.Tooltip("This pie chart displays the breakdown of transit methods used by commuters in Brooklyn. It represents the percentage of commuters using different modes of transportation, including public transit, car, walking, etc.", target="brooklyn-icon")
                ])
            ], style={'border': '2px solid green', 'margin-bottom': '10px'})
        ], width=12)
    ], style={'border': '2px black solid', 'padding': '10px', 'margin-top': '10px', 'margin-bottom': '20px', 'text-align': 'center'}),
    
    dbc.Row([
        dbc.Col([
            html.P("Source: U.S. Department of Transportation. (n.d.). Means of Transportation to Work. Retrieved May 11, 2023, from https://geodata.bts.gov/datasets/usdot::means-of-transportation-to-work/explore?location=19.863468%2C.314300%2C3.31&showTable=true", style={'font-size': '16px'}),
        ], width=12)
    ], style={'margin-bottom': '20px'})
])



# Run the app inside Jupyter notebook
app.run_server(mode='inline', debug=True, use_reloader=False, port = 8083)

Dash is running on http://127.0.0.1:8083/



INFO:dash.dash:Dash is running on http://127.0.0.1:8083/



<IPython.core.display.Javascript object>